# Constant expressions

A **constant expression** is an expression which can be evaluated during compilation. In C++03, it can only involve literal integers and constant variables which are themselves initialized using constant expressions.
- C++11 extends the possibilities to floats, functions and objects.
- C++17 adds the statement `if constexpr`, which is evaluated at compile time.
- C++20 adds `consteval`, which enforces a compile-time evaluation.

## `constexpr` variables

A variable declared as `constexpr` is implicitly` const`, and **must be evaluable at compile-time**. Unlike a `const` variable, it should not take up any space in memory, and its value should be directly substituted in the code wherever it is used. Unlike a `#define`, it is fully pre-evaluated by the C++ compiler, and will not suffer from the pitfalls specific to preprocessor macros.

Additionally, float values ​​are supported. We can now initialize, within a class definition, the member variables which are both `static` and` constexpr`. For example, this would not be allowed in old C++:

In [1]:
struct X
 {
  static constexpr float pi = 3.14 ;
 } ;

## `constexpr` functions

A function declared as `constexpr` **can optionally** be evaluated at compile time, if the arguments passed to it are themselves constant expressions. Thus, for example, we can call functions, during compilation, so to calculate the size of a fixed array:

In [1]:
constexpr int square( int n )
 { return n*n ; }

In [ ]:
#include <iostream>

In [ ]:
constexpr int t1 = 2 ;
int const t2 = 2 ;
int t3 = 2 ;
//...

In [11]:
double a1[square(t1)] = { 1., 2., 3., 4. } ;
double a2[square(t2)] = { 1., 2., 3., 4. } ;
double a3[square(t3)] = { 1., 2., 3., 4. } ;

input_line_24:4:11: error: variable-sized object may not be initialized
double a3[square(t3)] = { 1., 2., 3., 4. } ;
          ^~~~~~~~~~


Interpreter Error: 

Such functions were originally limited to a single `return` statement ! But all subsequent C++ versions have opened more and more the possibilities, so that today the main remaining prohibitions are:
- the function to be `virtual`,
- `try` blocks, `goto` statements,
- more than one `return `statement`,
- non-literal types for parameters and return value.

## `constexpr` constructor

The use of `constexpr` is allowed for a constructor if the body is empty and the members are explicitly initialized in the initialization area:

In [1]:
class Point
 {
  public :
    constexpr Point( int a_x, int a_y ) : m_x {a_x}, m_y {a_y} {}
  private :
    int m_x, m_y ;
 } ;

In [2]:
constexpr Point origin(0,0) ;

## `if constexpr` (C++17)

This variant of `if` is evaluated at compile time. The condition must obviously be a constant expression, evaluable at compilation time. This new *static if* is much better than a preprocessor `#if`, because its condition can be based on complex expressions evaluated by the compiler.

In this example, we are comparing two numbers, with a margin in the case of a floating point number.

In [1]:
#include <iostream>
#include <cmath>
#include <type_traits>
#include <limits>

In [3]:
template <class T>
bool equal( T lhs, T rhs )
 {
  if constexpr (std::is_floating_point_v<T>)
   { return (std::abs(lhs-rhs)<std::numeric_limits<T>::epsilon()) ; }
  else
   { return lhs == rhs ; }
 } 

In [11]:
std::cout
  <<"1. =~ .1+.1+.1+.1+.1+.1+.1+.1+.1+.1 ? "
  <<equal(1.,.1+.1+.1+.1+.1+.1+.1+.1+.1+.1)
  <<std::endl ;
std::cout<<"100 =~ 10*10 ? "<<equal(100,10*10)<<std::endl ;  

1. =~ .1+.1+.1+.1+.1+.1+.1+.1+.1+.1 ? 1
100 =~ 10*10 ? 1


**To be noticed**: the "if block" above is not compilable when `T` is `int`, because `std::numeric_limits<int>::epsilon()` does not exists. Yet, this is not a problem, since the compiler directly evaluates `std::is_floating_point_v<int>` to `false`, discards the "if block", and only compiles the "else block".

## `consteval` function (C++20)

Such a function can only be called at compile time. It is meant to be an implementation which comply to the rules of constant expressions, but is not as fast as what cane be done at runtime. 

## `constinit` variable (C++20)

This weaker variant of `constexpr` let you require a compile time evaluation, although the variable is not `const`and can be modified later on. This can only be used with global or static variables, and may help to solve the *Static Initialization Order Fiasco*.

Since you safely avoid such dangerous global or static variables, this is not your concern ;)

## `if consteval` (C++23)

This let you detect that the current code is evaluated at compile-time, and let you use the `consteval` function optimized for this. The `else` block, on the contrary, can use the function which is optimized for runtime evaluation.

# Take away

- A `const` variable **can** be initialized at compile-time.
- A `constexpr` variable **must** be initialized at compile-time, and is `const`.
- A `constinit` global/static variable **must** be initialized at compile-time, and is not `const`.

- A `constexpr` function call **can** be evaluated at compile-time, if its inputs are constant expressions.
- A `consteval` function call **must** be evaluated at compile-time.

- `if constexpr ()` is evaluated at compile-time.
- `if consteval` let you known if a compile-time evaluation is under way.

# Questions ?

# Exercise

The Fibonacci function is defined as follows:
* f(0) = 0
* f(1) = 1
* f(n) = f(n-1)+f(n-2)

It is written below using old meta-programming techniques, in order to be evaluated at compile time.
- Simplify it ?
- Mesure separately the time of compilation and excution.
- Change the type of `fibo10`, measure again and compare.

In [42]:
%%file tmp.constexpr.cpp

#include <iostream>

template<int N>
struct fibonacci
 {
  enum { value = fibonacci<N-1>::value + fibonacci<N-2>::value } ;
 } ;

template<>
struct fibonacci<1>
 {
  enum { value = 1 } ;
 } ;

template<>
struct fibonacci<0>
 {
  enum { value = 0 } ;
 } ;

int main()
 {
  constexpr int res {fibonacci(36)} ;
  std::cout<<res<<std::endl ;
  return 0 ;
 }

Overwriting tmp.constexpr.cpp


In [43]:
!rm -f tmp.constexpr.exe

In [44]:
!time g++ -std=c++17 tmp.constexpr.cpp -o tmp.constexpr.exe

2.01user 0.04system 0:02.05elapsed 99%CPU (0avgtext+0avgdata 64680maxresident)k
0inputs+48outputs (0major+17974minor)pagefaults 0swaps


In [45]:
!time ./tmp.constexpr.exe

14930352
0.11user 0.00system 0:00.12elapsed 99%CPU (0avgtext+0avgdata 3452maxresident)k
0inputs+0outputs (0major+143minor)pagefaults 0swaps


## Resources
* http://meetingcpp.com/blog/items/How-if-constexpr-simplifies-your-code-in-Cpp17.html
* https://www.codingame.com/playgrounds/2205/7-features-of-c17-that-will-simplify-your-code/constexpr-if
* https://linuxfr.org/news/cpp17-branche-a-la-compilation-if-constexpr
* https://solarianprogrammer.com/2017/12/27/cpp-17-constexpr-everything-as-much-as-the-compiler-can/

© *CNRS 2019*  
*This document was created by David Chamont and translated by Olga Abramkina. It is available under the [License Creative Commons - Attribution - No commercial use - Shared under the conditions 4.0 International](http://creativecommons.org/licenses/by-nc-sa/4.0/)*

